### Prompt, Response 1, and Response 2

In [22]:
# Preference data generation prompting

with open("../data/constitution_1.txt", "r") as f:
    constitution = f.read()

# num_data_points = 10 # could go up to 8000 for a single query
num_data_points = 10
domains = [
    "Health and Well-being",
    "Education and Learning",
    "Workplace and Employment",
    "Social Media and Online Behavior",
    "Environmental Sustainability",
    "Human Rights and Social Justice",
    "Technology and Privacy",
    "Family and Relationships",
    "Legal and Ethical Dilemmas",
    "Politics and Governance"
]


GENERATOR_PROMPT = \
    f"""{constitution}

    Given the constitution listed above, create a dataset with three columns: a prompt, Response 1, and Response 2. The prompt should describe a common scenario or situation or state in the world. Response 1 should obey the list of constitutions provided, ensuring it aligns with ethical principles like equality, respect, and fairness. Response 2 should be a counterexample that doesn't follow the constitutions and violates one or more of these principles. You dont need to start your prompt by saying "User asks". Ensure the prompts cover a variety of everyday situations only in f{domains[0]} context. Give me f{num_data_points} such examples.
    """

In [ ]:
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../.env")

client = OpenAI()

class PreferenceDataPoint(BaseModel):
    prompt: str
    response_1: str
    response_2: str

    def get_words(self):
        return len(self.prompt.split()) + len(self.response_1.split()) + len(self.response_2.split())

class PreferenceDataset(BaseModel):
    row: list[PreferenceDataPoint]

    def to_csv(self, path: str):
        with open(path, "w") as f:
            f.write("prompt,response_1,response_2\n")
            for row in self.row:
                response_1 = row.response_1.replace("Response 1:", "").strip()
                response_2 = row.response_2.replace("Response 2:", "").strip()
                f.write(f'"{row.prompt}","{response_1}","{response_2}"\n')

    def get_words(self):
        return sum([row.get_words() for row in self.row])

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": GENERATOR_PROMPT},
    ],
    response_format=PreferenceDataset,
)

event = completion.choices[0].message.parsed

### Cost Management

In [24]:
event.to_csv("../data/preference_data_1.csv")
output_tokens_for_10_datapoints = (event.get_words() * 100) / 75
print(f"Total tokens used for 10 data points: {output_tokens_for_10_datapoints}")
input_tokens = (len(GENERATOR_PROMPT.split()) * 100) / 75
print(f"Total tokens used for input: {input_tokens}")
output_cost_for_10_gen = (output_tokens_for_10_datapoints * 0.6) / 1000000
input_cost = (input_tokens * 0.15) / 1000000
total_cost = output_cost_for_10_gen + input_cost
print(f"Total cost for input w. 10 outputs and 1/10 context: ${total_cost}")
print(f"Total cost for input w. 1000 outputs and 1/10 context: ${input_cost + (output_cost_for_10_gen * 100)}")
print(f"Total cost for input w. 10000 outputs and 1/10 context: ${input_cost + (output_cost_for_10_gen * 1000)}")


Total tokens used for 10 data points: 832.0
Total tokens used for input: 1346.6666666666667
Total cost for input w. 10 outputs and 1/10 context: $0.0007012
Total cost for input w. 1000 outputs and 1/10 context: $0.050122
Total cost for input w. 10000 outputs and 1/10 context: $0.49940199999999996


In [14]:
# given that there is $0.5 for around 8000 outputs for one context and one query, we do this 2 times with higher temperature per each context giving 20 * 8000 outputs with $10 for the cost.


# since max output tokens for the model is 16000, and we want to generate 800000 output tokens:
num_runs_per_context_and_half_query = 800000 / 16000
print(f"Number of runs per context and half query: {num_runs_per_context_and_half_query}")
output_cost_for_160_gen = (16000 * 0.6) / 1000000
total_cost_for_one_context = num_runs_per_context_and_half_query * (input_cost + output_cost_for_160_gen)
print(f"Total cost for one context: ${total_cost_for_one_context}")

Number of runs per context and half query: 50.0
Total cost for one context: $0.48999


### Preference rating by Llama 3

In [ ]:
# file error checking
import pandas as pd
import os

const_number = 1
direct = f"../data/preferences/{const_number}"

all_files = os.listdir(direct)
dataframes = []
for file in all_files:
    file_path = os.path.join(direct, file)
    try:
        df = pd.read_csv(file_path)
        dataframes.append(df)
    except pd.errors.ParserError as e:
        print(f"Parsing error in file {file_path}: {e}")

In [3]:
# compile all csv files into one
import pandas as pd
import os

NUM_runs=200
NUM_classes=10
const_number=1

direct = f"../data/preferences/{const_number}"
# for all files in the dir, compile into one csv with columns, prompt, response_1, response_2
all_files = os.listdir(direct)
df = pd.concat([pd.read_csv(f"{direct}/{file}") for file in all_files])
df.info()
df = df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 35842 entries, 0 to 16
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prompt      35842 non-null  object
 1   response_1  35842 non-null  object
 2   response_2  35842 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [ ]:
from dotenv import load_dotenv
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import numpy as np
import multiprocessing as mp
import pandas as pd

load_dotenv("../.env")

model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# evaluate function
def evaluate_preference_batch(rows, constitution, pipe):
    prompts = []
    for row in rows:
        prompt = row['prompt']
        response_1 = row['response_1']
        response_2 = row['response_2']

        # Construct the input text
        input_text = f"""{constitution}
        Given the constitution above and the following prompt:

        "{prompt}"

        Here are two responses:

        Response 1: "{response_1}"

        Response 2: "{response_2}"

        Based on the constitution, which response better adheres to the guidelines? Respond with "0" if Response 1 is better or "1" if Response 2 is better."""
        prompts.append(input_text)

    # Generate the model's responses in batch
    outputs = pipe(prompts, max_new_tokens=10, return_full_text=False)

    preferences = []
    for output in outputs:
        generated_text = output['generated_text']
        preference = ''.join(filter(str.isdigit, generated_text.strip()))
        if preference in ['0', '1']:
            preferences.append(int(preference))
        else:
            preferences.append(None)
    
    return preferences

# run inference
def worker(device_id, df_chunk, constitution, return_dict):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(device_id)
    from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

    # Load the model and tokenizer within the process
    model_id = "meta-llama/Llama-3.2-3B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto",
    )

    # Set up the pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )

    batch_size = 8  # Adjust the batch size as needed
    preferences = []
    for i in range(0, len(df_chunk), batch_size):
        batch = df_chunk.iloc[i:i+batch_size]
        batch_preferences = evaluate_preference_batch(batch.to_dict('records'), constitution, pipe)
        preferences.extend(batch_preferences)

    df_chunk = df_chunk.copy()
    df_chunk['preference'] = preferences
    return_dict[device_id] = df_chunk

# Split the DataFrame into chunks for each GPU
num_gpus = 2
df_chunks = np.array_split(df, num_gpus)

with open("../data/constitution_1.txt", "r") as f:
    constitution = f.read()

manager = mp.Manager()
return_dict = manager.dict()
processes = []
for i in range(num_gpus):
    p = mp.Process(target=worker, args=(i, df_chunks[i], constitution, return_dict))
    processes.append(p)
    p.start()

for p in processes:
    p.join()

# Combine the results
df_results = pd.concat(return_dict.values(), ignore_index=True)
print(df_results)

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2
Python-dotenv could not parse statement starting at line 5
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.33s/it]
/home/mila/s/shahrad.mohammadzadeh/projects/olmo_continual/.venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.36it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

: 